In [ ]:
!pip install flask requests transformers

In [ ]:
%%writefile app.py
from flask import Flask, request, jsonify
from transformers import pipeline

app = Flask(__name__)

# Pre-made responses
responses = {
    "hello": "Hi there! How can I help you today?",
    "bye": "Goodbye! Have a nice day.",
    "default": "I'm not sure how to respond to that."
}

# Initialize sentiment analysis pipelines for two different models


def analyze_sentiment(message, model="distil"):
    """
    Analyze the sentiment of the message using the selected model.

    Args:
        message (str): The text to analyze.
        model (str): 'distil' or 'nlptown'

    Returns:
        dict: A dictionary with the sentiment analysis result.
    """


@app.route('/chat', methods=['POST'])
def chat():
    data = request.get_json()
    user_message = data.get("message", "")
    # Add model chopice

    sentiment_result = None

    # Decide if the sentiment is negative (i.e., angry)
    angry = False

    if angry:
        reply = "I sense some anger in your words. Let's try to keep the conversation calm."
    else:
        # Use basic rule-based matching for greetings and goodbyes
        if "hello" in user_message.lower():
            reply = responses.get("hello")
        elif "bye" in user_message.lower():
            reply = responses.get("bye")
        else:
            reply = responses.get("default")

    return jsonify({
        "reply": reply,
        "sentiment_analysis": sentiment_result
    })

if __name__ == '__main__':
    # The server will run on 0.0.0.0:5000
    app.run(host='0.0.0.0', port=5000)


In [ ]:
!nohup python app.py &


In [ ]:
import socket
import time

def check_port(host, port):
  port_closed = True
  while port_closed:
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        try:
            sock.connect((host, port))
            print(f"Port {port} on {host} is open!")
            port_closed = False
        except ConnectionRefusedError:
            print(f"Port {port} on {host} is closed.")
            time.sleep(0.5)


check_port("127.0.0.1", 5000)

In [ ]:
import requests

url = "http://127.0.0.1:5000/chat"

# Test messages along with a choice of sentiment model.
# Use "distil" for the distilbert model and "nlptown" for the nlptown model.
test_messages = [
    {"message": "Hello there!", "model": "distil"},
    {"message": "I hate this so much!", "model": "distil"},
    {"message": "This is absolutely terrible!", "model": "nlptown"},
    {"message": "Goodbye", "model": "nlptown"}
]

for payload in test_messages:
    response = requests.post(url, json=payload)
    if response.ok:
        result = response.json()
        print(f"User: {payload['message']} (Model: {payload['model']})")
        print(f"Chatbot: {result['reply']}")
        print(f"Sentiment: {result['sentiment_analysis']}")
        print("-" * 50)
    else:
        print("Failed to connect to the chatbot service.")
